In [68]:
from elasticsearch import Elasticsearch
import json
elastic_client = Elasticsearch(hosts=["http://143.167.8.152:9300"], http_auth=('students', 'foxtrot'))

def random_rumours(amount, categories):    
    
#     #using hashtags
#     public_authority_actions = []
#     community_impact = ["filmyourhospital", "filmyourhospitals", "endthelockdown", "endthelockdownuk", "reopenbritain"]
#     medical_advice = ["coronavirusfacts",  "coronafacts", "covid19symptoms", "covidsymptoms", "hydroxichloroquine", "hydroxychloroquine"]
#     claims_about_prominent_actors = ["gatesvirus", "sorosvirus", "coronavillains", "kungflu", "wholiedpeopledied", "chinaliedandpeopledied",  "chinaliedpeopledied", "chinaliespeopledied", "israelvirus", "americavirus", "coronaviruscoverup", "chinesevirus", "wuhanvirus", "ccpvirus"]
#     conspiracy_theories = ["5g",  "5gcoronavirus", "coronavirus5g", "chinesebioterrorism", "gatesvirus", "ciavirus", "deepstatevirus", "preventnwo",  "nwoevilelites", "nwovirus", "nwo", "greatreset", "coronacon",   "coronafakenews",  "coronafraud", "coronahoax", "scamdemic", "plandemic", "filmyourhospital", "filmyourhospitals", "coronaviruscoverup", "coronavillains", "depopulation", "nwoevilplans"]
#     virus_transmission = ["coronavirusfacts",  "coronafacts"]
#     virus_origin = ["coronavirusfacts",  "coronafacts", "covid19symptoms", "covidsymptoms", "coronasymptoms"]
#     civil_disobedience = ["endthelockdown", "endthelockdownuk", "reopenbritain", "resistthegreatreset", "idonotconsent", "covidiots", "coronaviruscoverup"]
#     vaccines = ["coronavirusfacts",  "coronafacts", "astrazeneca",  "oxfordvaccine"]
#     other = ["coronabollocks", "coronacon",   "coronafakenews",  "coronafraud", "coronahoax", "cronyvirus", "scamdemic", "plandemic"]
    
#     def f(x):
#         return {
#             "Public authority actions, policy, and communications": (public_authority_actions, float(x[1])),
#             "Community spread and impact": (community_impact, float(x[1])),
#             "Medical advice and self-treatments": (medical_advice, float(x[1])),
#             "Claims about prominent actors": (claims_about_prominent_actors, float(x[1])),
#             "Conspiracy theories": (conspiracy_theories, float(x[1])),
#             "Virus transmission": (virus_transmission, float(x[1])),
#             "Virus origin and properties": (virus_origin, float(x[1])),
#             "Public preparedness, protests, and civil disobedience": (civil_disobedience, float(x[1])),
#             "Vaccines, medical treatments, and tests": (vaccines, float(x[1])),
#             "Other": (other, float(x[1])),
#         }[x[0]]

    
    should_array = []
    must_array = [{"match": {"is_a_retweet": "false"}}, {"match": {"is_a_quote": "false"}}]
    #for x in categories:
     #   query = f(x)
      #  should_array.append({"terms": {"hashtags": query[0], "boost": query[1]}})
        
        
    input_file = open("bowPerCategory.json", "r")
    input_data = input_file.read()
    #using BOW
    category_BOW = json.loads(input_data) #this is a dict, so category_BOW["Virus origin and properties"] = array of words
    
    
    should_array = []
    for x in categories:
        for y in category_BOW[x]: #input must be exactly the same as keys used in json
            should_array.append({"match": {"tweet_text": "*"+y+"*"}}) 
            #this will change based on how the json is parsed - y[0] just needs to be the text, and y[1] the number of times that
            #word appears in the bag. x[0] is the category input to the method, and x[1] is the number input to the method for that
            #category (user boost)
            
    
    query_body = {
      "query": {
        "bool": {
          "must": must_array,
          "must_not": [
            {"match": {"hashtags": "obamagate"}},
            {"exists": {"field": "in_reply_to_user_id"}},
            {"exists": {"field": "in_reply_to_status_id"}},
            {"exists": {"field": "in_reply_to_screen_name"}},
            {"match": {"message": "*media_url*"}}
          ],
          "should": should_array
        }
      },
      "collapse" : {
        "field" : "tweet_id.keyword"
      }
    }

    
    result = elastic_client.search(index="covid19_misinfo_index", body=query_body, size=amount, request_timeout=300000, _source="tweet_text,tweet_id")

    all_hits = result['hits']['hits']
    
    tweets = []
    for num, doc in enumerate(all_hits):
        tweets.append([doc["_source"]["tweet_text"], doc["_source"]["tweet_id"]])
    return tweets    

In [71]:
print(random_rumours(3, ["Community spread and impact", "Virus transmission"]))

TransportError: TransportError(504, '<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx/1.17.10</center>\r\n</body>\r\n</html>\r\n')

In [3]:
#CATEGORIES:
#Virus origin and properties
#Virus transmission
#Vaccines, medical treatments, and tests
#Medical advice and self-treatments
#Other
#Public preparedness, protests, and civil disobedience
#Public authority actions, policy, and communications
#Community spread and impact
#Conspiracy theories
#Claims about prominent actors

SyntaxError: invalid syntax (<ipython-input-3-effef6a475c0>, line 1)